In [1]:
from google.colab import drive
drive.mount("/content/drive")

from pathlib import Path
import numpy as np, pandas as pd, random

BASE = Path("/content/drive/MyDrive/biolip_gnn")
GRAPH_DIR = BASE / "graphs_labeled_v6_feat837"
OUT_DIR = BASE / "out"
OUT_DIR.mkdir(parents=True, exist_ok=True)

npz_files = sorted(GRAPH_DIR.glob("*.npz"))
print("graphs =", len(npz_files))
assert len(npz_files) == 837, f"Expected 837 graphs, found {len(npz_files)}. Check GRAPH_DIR."

def split_paths(paths, seed=42):
    paths = paths.copy()
    random.Random(seed).shuffle(paths)
    n = len(paths)
    ntr = int(0.70*n)
    nva = int(0.15*n)
    return paths[:ntr], paths[ntr:ntr+nva], paths[ntr+nva:]

seeds = [1, 7, 42, 123, 999]

# show split sizes for one seed
tr, va, te = split_paths(npz_files, seed=42)
print("seed=42 split sizes:", "train", len(tr), "val", len(va), "test", len(te))


Mounted at /content/drive
graphs = 837
seed=42 split sizes: train 585 val 125 test 127


In [6]:
import torch

torch_ver = torch.__version__.split('+')[0]
cuda_tag = "cu121" if torch.version.cuda else "cpu"

# Install the compiled deps (no pyg-lib)
!pip -q install torch-scatter torch-sparse torch-cluster torch-spline-conv \
  -f https://data.pyg.org/whl/torch-{torch_ver}+{cuda_tag}.html

# Install torch_geometric itself
!pip -q install torch-geometric


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 423, in run
    _, build_failures = build(
                        ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/wheel_builder.py", line 319, in build
    wheel_file = _build_one(
                 ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/wheel_builder.py", line 193, in _build_one
    wheel_path = _build

In [7]:
# Node feature construction (baseline + feat_v1)

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data

def load_npz(p):
    z = np.load(p, allow_pickle=True)
    return {k: z[k] for k in z.files}

# Kyte–Doolittle hydrophobicity
KD = {
    "A": 1.8, "C": 2.5, "D": -3.5, "E": -3.5, "F": 2.8,
    "G": -0.4, "H": -3.2, "I": 4.5, "K": -3.9, "L": 3.8,
    "M": 1.9, "N": -3.5, "P": -1.6, "Q": -3.5, "R": -4.5,
    "S": -0.8, "T": -0.7, "V": 4.2, "W": -0.9, "Y": -1.3
}

# charge: -1 (D,E), +1 (K,R,H), 0 otherwise
CHARGE = {aa: 0 for aa in KD}
for aa in ["D","E"]:
    CHARGE[aa] = -1
for aa in ["K","R","H"]:
    CHARGE[aa] = +1

# polarity (simple binary)
POLAR = {aa: 0 for aa in KD}
for aa in ["S","T","N","Q","Y","C","W","H","K","R","D","E"]:
    POLAR[aa] = 1

# IMPORTANT: map x_idx -> 1-letter AA.
# Your dataset likely used 20 AAs + X/UNK. We'll support both.
AA_LIST = list("ACDEFGHIKLMNPQRSTVWY")  # 20
IDX_TO_AA = {i: aa for i, aa in enumerate(AA_LIST)}
IDX_TO_AA[20] = "X"  # common UNK index

def compute_degree(edge_index, N):
    # edge_index shape (2, E)
    src = edge_index[0]
    deg = torch.zeros(N, dtype=torch.float)
    deg.scatter_add_(0, src, torch.ones(src.shape[0], dtype=torch.float))
    return deg

def compute_mean_neighbor_dist(edge_index, edge_dist, N):
    # mean dist over edges leaving each node, ignoring sequence edges where dist==0
    src = edge_index[0]
    dist = edge_dist.clone().float()

    mask = dist > 0
    src2 = src[mask]
    dist2 = dist[mask]

    sumd = torch.zeros(N, dtype=torch.float)
    cnt  = torch.zeros(N, dtype=torch.float)

    sumd.scatter_add_(0, src2, dist2)
    cnt.scatter_add_(0, src2, torch.ones_like(dist2))

    mean = sumd / (cnt + 1e-9)

    # nodes with no spatial neighbors: set to global mean (or 0)
    global_mean = dist2.mean().item() if dist2.numel() > 0 else 8.0
    mean[cnt == 0] = global_mean
    return mean

def npz_to_data_baseline(g):
    edge_index = torch.tensor(g["edge_index"], dtype=torch.long)
    y = torch.tensor(g["y"], dtype=torch.long)
    aa_idx = torch.tensor(g["x_idx"], dtype=torch.long)  # shape (N,)
    N = aa_idx.numel()

    deg = compute_degree(edge_index, N)
    deg = (deg - deg.mean()) / (deg.std() + 1e-9)

    # x: [aa_idx, degree]
    x = torch.cat([aa_idx.view(-1,1).float(), deg.view(-1,1)], dim=1)
    return Data(x=x, edge_index=edge_index, y=y)

def npz_to_data_feat_v1(g):
    edge_index = torch.tensor(g["edge_index"], dtype=torch.long)
    edge_dist  = torch.tensor(g["edge_dist"], dtype=torch.float)
    y = torch.tensor(g["y"], dtype=torch.long)

    aa_idx = torch.tensor(g["x_idx"], dtype=torch.long)
    N = aa_idx.numel()

    # degree
    deg = compute_degree(edge_index, N)
    deg = (deg - deg.mean()) / (deg.std() + 1e-9)

    # mean neighbor dist
    mean_dist = compute_mean_neighbor_dist(edge_index, edge_dist, N)
    mean_dist = (mean_dist - mean_dist.mean()) / (mean_dist.std() + 1e-9)

    # sequence position i/(L-1)
    L = float(max(N-1, 1))
    seq_pos = torch.arange(N, dtype=torch.float) / L
    seq_pos = (seq_pos - seq_pos.mean()) / (seq_pos.std() + 1e-9)

    # physchem from aa_idx -> aa -> numbers
    aa_chars = [IDX_TO_AA.get(int(i), "X") for i in aa_idx.cpu().numpy().tolist()]
    hydro = torch.tensor([KD.get(a, 0.0) for a in aa_chars], dtype=torch.float)
    charge = torch.tensor([CHARGE.get(a, 0.0) for a in aa_chars], dtype=torch.float)
    polar = torch.tensor([POLAR.get(a, 0.0) for a in aa_chars], dtype=torch.float)

    # normalize continuous features
    hydro = (hydro - hydro.mean()) / (hydro.std() + 1e-9)
    charge = (charge - charge.mean()) / (charge.std() + 1e-9)
    polar = (polar - polar.mean()) / (polar.std() + 1e-9)

    # x: [aa_idx, degree, mean_dist, seq_pos, hydro, charge, polar]
    x = torch.cat([
        aa_idx.view(-1,1).float(),
        deg.view(-1,1),
        mean_dist.view(-1,1),
        seq_pos.view(-1,1),
        hydro.view(-1,1),
        charge.view(-1,1),
        polar.view(-1,1),
    ], dim=1)

    return Data(x=x, edge_index=edge_index, y=y)

# quick sanity on one graph
g0 = load_npz(npz_files[0])
d0b = npz_to_data_baseline(g0)
d0f = npz_to_data_feat_v1(g0)
print("baseline x shape:", d0b.x.shape, "| feat_v1 x shape:", d0f.x.shape)

baseline x shape: torch.Size([315, 2]) | feat_v1 x shape: torch.Size([315, 7])


In [8]:
# STEP 3: Train + evaluate baseline vs feat_v1

!pip -q install torch_geometric -U
from torch_geometric.loader import DataLoader
from torch_geometric.nn import SAGEConv
from sklearn.metrics import precision_recall_curve, average_precision_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

@torch.no_grad()
def collect_probs(model, loader):
    model.eval()
    P, Y = [], []
    for b in loader:
        b = b.to(device)
        p = torch.sigmoid(model(b)).detach().cpu().numpy()
        y = b.y.detach().cpu().numpy()
        P.append(p); Y.append(y)
    return np.concatenate(P), np.concatenate(Y)

def thr_maxf1(probs, y):
    prec, rec, thr = precision_recall_curve(y, probs)
    f1 = (2*prec[:-1]*rec[:-1])/(prec[:-1]+rec[:-1]+1e-9)
    return float(thr[int(np.argmax(f1))])

def thr_precision_target(probs, y, target=0.20):
    prec, rec, thr = precision_recall_curve(y, probs)
    prec2, rec2, thr2 = prec[:-1], rec[:-1], thr
    ok = np.where(prec2 >= target)[0]
    if len(ok)==0:
        return None
    best = ok[np.argmax(rec2[ok])]
    return float(thr2[best])

def prf_at_thr(probs, y, thr):
    pred = (probs >= thr).astype(int)
    tp = int(((pred==1) & (y==1)).sum())
    fp = int(((pred==1) & (y==0)).sum())
    fn = int(((pred==0) & (y==1)).sum())
    prec = tp/(tp+fp+1e-9)
    rec  = tp/(tp+fn+1e-9)
    f1   = 2*prec*rec/(prec+rec+1e-9)
    return prec, rec, f1

def compute_pos_weight(graphs):
    pos = sum(int(d.y.sum()) for d in graphs)
    tot = sum(int(d.y.numel()) for d in graphs)
    neg = tot - pos
    return torch.tensor([neg/max(pos,1)], dtype=torch.float)

class SAGE_NodeClassifier(nn.Module):
    def __init__(self, num_aa=21, emb_dim=32, hidden=64, extra_feats=1):
        super().__init__()
        self.emb = nn.Embedding(num_aa, emb_dim)
        in_dim = emb_dim + extra_feats
        self.conv1 = SAGEConv(in_dim, hidden)
        self.conv2 = SAGEConv(hidden, hidden)
        self.lin1  = nn.Linear(hidden, hidden)
        self.lin2  = nn.Linear(hidden, 1)

    def forward(self, data):
        aa = data.x[:,0].long()
        xf = data.x[:,1:].float()
        h = self.emb(aa)
        h = torch.cat([h, xf], dim=1)         # safe
        h = torch.relu(self.conv1(h, data.edge_index))
        h = torch.relu(self.conv2(h, data.edge_index))
        h = torch.relu(self.lin1(h))
        return self.lin2(h).squeeze(-1)

def train_model(train_graphs, extra_feats, epochs=6):
    loader = DataLoader(train_graphs, batch_size=4, shuffle=True)
    model = SAGE_NodeClassifier(extra_feats=extra_feats).to(device)
    crit = nn.BCEWithLogitsLoss(pos_weight=compute_pos_weight(train_graphs).to(device))
    opt = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    for _ in range(epochs):
        model.train()
        for b in loader:
            b = b.to(device)
            loss = crit(model(b), b.y.float())
            opt.zero_grad(); loss.backward(); opt.step()
    return model

rows = []
for sd in seeds:
    tr, va, te = split_paths(npz_files, sd)

    # loaders for baseline
    train_b = [npz_to_data_baseline(load_npz(p)) for p in tr]
    val_b   = [npz_to_data_baseline(load_npz(p)) for p in va]
    test_b  = [npz_to_data_baseline(load_npz(p)) for p in te]

    val_loader_b  = DataLoader(val_b, batch_size=4, shuffle=False)
    test_loader_b = DataLoader(test_b, batch_size=4, shuffle=False)

    m_b = train_model(train_b, extra_feats=1, epochs=6)
    vprob, vy = collect_probs(m_b, val_loader_b)
    tprob, ty = collect_probs(m_b, test_loader_b)

    thr_f1 = thr_maxf1(vprob, vy)
    thr_p20 = thr_precision_target(vprob, vy, 0.20)
    thr_p15 = thr_precision_target(vprob, vy, 0.15)

    row = {"seed": sd, "model":"baseline", "n_graphs": len(npz_files),
           "test_auprc": float(average_precision_score(ty, tprob))}
    P,R,f1 = prf_at_thr(tprob, ty, thr_f1)
    row.update({"test_P_maxF1":P, "test_R_maxF1":R, "test_F1_maxF1":f1})

    if thr_p20 is None:
        row.update({"test_P_p20":np.nan,"test_R_p20":np.nan,"test_F1_p20":np.nan,"p20_fallback":True})
    else:
        P,R,f1 = prf_at_thr(tprob, ty, thr_p20)
        row.update({"test_P_p20":P,"test_R_p20":R,"test_F1_p20":f1,"p20_fallback":False})

    if thr_p15 is None:
        row.update({"test_P_p15":np.nan,"test_R_p15":np.nan,"test_F1_p15":np.nan,"p15_fallback":True})
    else:
        P,R,f1 = prf_at_thr(tprob, ty, thr_p15)
        row.update({"test_P_p15":P,"test_R_p15":R,"test_F1_p15":f1,"p15_fallback":False})

    rows.append(row)

    # loaders for feat_v1
    train_f = [npz_to_data_feat_v1(load_npz(p)) for p in tr]
    val_f   = [npz_to_data_feat_v1(load_npz(p)) for p in va]
    test_f  = [npz_to_data_feat_v1(load_npz(p)) for p in te]

    val_loader_f  = DataLoader(val_f, batch_size=4, shuffle=False)
    test_loader_f = DataLoader(test_f, batch_size=4, shuffle=False)

    m_f = train_model(train_f, extra_feats=6, epochs=6)  # 6 numeric feats beyond aa_idx
    vprob, vy = collect_probs(m_f, val_loader_f)
    tprob, ty = collect_probs(m_f, test_loader_f)

    thr_f1 = thr_maxf1(vprob, vy)
    thr_p20 = thr_precision_target(vprob, vy, 0.20)
    thr_p15 = thr_precision_target(vprob, vy, 0.15)

    row = {"seed": sd, "model":"feat_v1", "n_graphs": len(npz_files),
           "test_auprc": float(average_precision_score(ty, tprob))}
    P,R,f1 = prf_at_thr(tprob, ty, thr_f1)
    row.update({"test_P_maxF1":P, "test_R_maxF1":R, "test_F1_maxF1":f1})

    if thr_p20 is None:
        row.update({"test_P_p20":np.nan,"test_R_p20":np.nan,"test_F1_p20":np.nan,"p20_fallback":True})
    else:
        P,R,f1 = prf_at_thr(tprob, ty, thr_p20)
        row.update({"test_P_p20":P,"test_R_p20":R,"test_F1_p20":f1,"p20_fallback":False})

    if thr_p15 is None:
        row.update({"test_P_p15":np.nan,"test_R_p15":np.nan,"test_F1_p15":np.nan,"p15_fallback":True})
    else:
        P,R,f1 = prf_at_thr(tprob, ty, thr_p15)
        row.update({"test_P_p15":P,"test_R_p15":R,"test_F1_p15":f1,"p15_fallback":False})

    rows.append(row)

df = pd.DataFrame(rows)
display(df)

metrics = ["test_auprc","test_P_maxF1","test_R_maxF1","test_F1_maxF1","test_P_p20","test_R_p20","test_F1_p20","test_P_p15","test_R_p15","test_F1_p15"]
summary = df.groupby("model")[metrics].agg(["mean","std"])
display(summary)

save_path = OUT_DIR / "day16_feature_report.csv"
df.to_csv(save_path, index=False)
print("Saved:", save_path)


Device: cpu


,seed,model,n_graphs,test_auprc,test_P_maxF1,test_R_maxF1,test_F1_maxF1,test_P_p20,test_R_p20,test_F1_p20,p20_fallback,test_P_p15,test_R_p15,test_F1_p15,p15_fallback
0,1,baseline,837,0.082081,0.119488,0.163743,0.138158,0.133846,0.084795,0.103819,False,0.115707,0.201754,0.147069,False
1,1,feat_v1,837,0.095134,0.113706,0.327485,0.168802,0.152130,0.146199,0.149105,False,0.121099,0.283626,0.169729,False
2,7,baseline,837,0.082937,0.095652,0.313390,0.146569,0.155844,0.011396,0.021239,False,0.142466,0.098765,0.116657,False
3,7,feat_v1,837,0.095827,0.121884,0.250712,0.164026,0.176806,0.088319,0.117796,False,0.127871,0.232669,0.165039,False
4,42,baseline,837,0.092417,0.119459,0.212705,0.152994,0.184697,0.067372,0.098731,False,0.126034,0.190568,0.151724,False
5,42,feat_v1,837,0.097964,0.110555,0.297401,0.161189,0.182773,0.083734,0.114851,False,0.132948,0.221367,0.166125,False
6,123,baseline,837,0.100066,0.110802,0.300094,0.161847,0.190647,0.099718,0.130945,False,0.133292,0.202258,0.160688,False
7,123,feat_v1,837,0.105209,0.137024,0.186265,0.157895,0.195960,0.091251,0.124519,False,0.135637,0.199436,0.161462,False
8,999,baseline,837,0.105707,0.145606,0.245399,0.182768,NaN,NaN,NaN,True,0.186170,0.092025,0.123167,False
9,999,feat_v1,837,0.118969,0.148242,0.255039,0.187500,0.280992,0.029798,0.053883,False,0.244755,0.092025,0.133758,False


test_auprc           test_P_maxF1           test_R_maxF1            \
               mean       std         mean       std         mean       std   
model                                                                         
baseline   0.092642  0.010387     0.118202  0.018148     0.247066  0.061922   
feat_v1    0.102621  0.009973     0.126282  0.015988     0.263381  0.053488   

         test_F1_maxF1           test_P_p20           test_R_p20            \
                  mean       std       mean       std       mean       std   
model                                                                        
baseline      0.156467  0.017073   0.166259  0.026418    0.06582  0.038616   
feat_v1       0.167882  0.011674   0.197732  0.049185    0.08786  0.041241   

         test_F1_p20           test_P_p15           test_R_p15            \
                mean       std       mean       std       mean       std   
model                                                                      
baseline    0.088683  0.047134   0.140734  0.027227   0.157074  0.056549   
feat_v1     0.112031  0.035188   0.152462  0.051890   0.205824  0.070722   

         test_F1_p15            
                mean       std  
model                           
baseline    0.139861  0.018997  
feat_v1     0.159223  0.014537

Saved: /content/drive/MyDrive/biolip_gnn/out/day16_feature_report.csv


In [9]:
# Top-K metrics on test proteins (seed=42 by default)

def precision_recall_at_k(probs, y, k):
    k = min(k, len(y))
    idx = np.argsort(-probs)[:k]
    tp = int(y[idx].sum())
    prec = tp / (k + 1e-9)
    rec = tp / (int(y.sum()) + 1e-9)
    return prec, rec

def eval_topk_for_seed(model, test_graphs, k_list=[5,10,20]):
    loader = DataLoader(test_graphs, batch_size=1, shuffle=False)
    all_rows = []
    model.eval()
    with torch.no_grad():
        for b in loader:
            b = b.to(device)
            probs = torch.sigmoid(model(b)).cpu().numpy()
            y = b.y.cpu().numpy()

            for k in k_list:
                p, r = precision_recall_at_k(probs, y, k)
                all_rows.append({"K": k, "prec": p, "rec": r})
    dfk = pd.DataFrame(all_rows).groupby("K")[["prec","rec"]].mean().reset_index()
    return dfk

# re-train for seed=42 just for top-k (keeps it consistent)
sd = 42
tr, va, te = split_paths(npz_files, sd)

test_b  = [npz_to_data_baseline(load_npz(p)) for p in te]
test_f  = [npz_to_data_feat_v1(load_npz(p)) for p in te]

m_b = train_model([npz_to_data_baseline(load_npz(p)) for p in tr], extra_feats=1, epochs=6)
m_f = train_model([npz_to_data_feat_v1(load_npz(p)) for p in tr], extra_feats=6, epochs=6)

topk_b = eval_topk_for_seed(m_b, test_b)
topk_f = eval_topk_for_seed(m_f, test_f)

topk_b["model"] = "baseline"
topk_f["model"] = "feat_v1"
topk = pd.concat([topk_b, topk_f], ignore_index=True)
display(topk)

save_path = OUT_DIR / "day16_topk_report.csv"
topk.to_csv(save_path, index=False)
print("Saved:", save_path)

,K,prec,rec,model
0,5,0.152756,0.105173,baseline
1,10,0.135433,0.176440,baseline
2,20,0.117323,0.296957,baseline
3,5,0.181102,0.118372,feat_v1
4,10,0.161417,0.204682,feat_v1
5,20,0.128346,0.315875,feat_v1


Saved: /content/drive/MyDrive/biolip_gnn/out/day16_topk_report.csv


In [11]:
# Save-only top20 visualization for 2 test proteins (seed=42)

!pip install Biopython
import gzip
import matplotlib.pyplot as plt
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB.Polypeptide import is_aa

STRUCT_DIR = BASE / "structures"
PLOT_DIR = OUT_DIR / "day16_pred_plots"
PLOT_DIR.mkdir(parents=True, exist_ok=True)

parser = MMCIFParser(QUIET=True)

def load_structure_from_cif_gz(pdb_id: str):
    path = STRUCT_DIR / f"{pdb_id.lower()}.cif.gz"
    if not path.exists():
        return None
    with gzip.open(path, "rt") as handle:
        return parser.get_structure(pdb_id.lower(), handle)

def chain_ca_map(structure, chain_id: str):
    if structure is None:
        return {}
    model = structure[0]
    if chain_id not in model:
        return {}
    m = {}
    for res in model[chain_id]:
        if not is_aa(res, standard=False):
            continue
        if "CA" not in res:
            continue
        rid = res.get_id()  # (' ', resseq, icode)
        resnum = int(rid[1])
        icode = rid[2].strip()
        if icode != "":
            continue
        m[resnum] = res["CA"].get_coord().astype(float)
    return m

def save_topk_plot(npz_path, model, tag, topk=20, use_feat=False):
    z = np.load(npz_path, allow_pickle=True)
    pdb_id = str(z["pdb_id"])
    chain  = str(z["chain"])
    resseq = z["resseq"].astype(int)
    y_true = z["y"].astype(int)

    structure = load_structure_from_cif_gz(pdb_id)
    cmap = chain_ca_map(structure, chain)
    if len(cmap) == 0:
        print("No coords:", npz_path.name)
        return None

    coords, keep = [], []
    for i, r in enumerate(resseq):
        if int(r) in cmap:
            coords.append(cmap[int(r)])
            keep.append(i)
    coords = np.vstack(coords)
    keep = np.array(keep, dtype=int)

    g = load_npz(npz_path)
    data = npz_to_data_feat_v1(g) if use_feat else npz_to_data_baseline(g)
    data = data.to(device)

    model.eval()
    with torch.no_grad():
        probs = torch.sigmoid(model(data)).cpu().numpy()

    yk = y_true[keep]
    pk = probs[keep]

    k = min(topk, len(pk))
    top_idx = np.argsort(-pk)[:k]
    hits = int(yk[top_idx].sum())

    fig = plt.figure(figsize=(6.5, 6))
    ax = fig.add_subplot(111, projection="3d")

    # all residues (faint)
    ax.scatter(coords[:,0], coords[:,1], coords[:,2], s=6, alpha=0.12)

    # true positives (orange-ish default)
    if (yk==1).any():
        c = coords[yk==1]
        ax.scatter(c[:,0], c[:,1], c[:,2], s=35, alpha=0.95)

    # predicted top-k (green-ish default)
    c = coords[top_idx]
    ax.scatter(c[:,0], c[:,1], c[:,2], s=18, alpha=0.95)

    ax.set_title(f"{npz_path.stem} | {tag} | hits_in_top{topk}={hits}/{k}")
    ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")

    outp = PLOT_DIR / f"{npz_path.stem}_{tag}_top{topk}.png"
    fig.savefig(outp, dpi=220, bbox_inches="tight")
    plt.close(fig)
    return outp

# pick 2 test proteins from seed=42 split
sd = 42
tr, va, te = split_paths(npz_files, sd)
two = random.sample(te, 2)
print("Chosen test graphs:", [p.name for p in two])

# Use the models m_b and m_f already trained in Step 4
saved = []
for p in two:
    saved.append(save_topk_plot(p, m_b, "baseline", topk=20, use_feat=False))
    saved.append(save_topk_plot(p, m_f, "feat_v1", topk=20, use_feat=True))

# sanity check: list files
pngs = sorted(PLOT_DIR.glob("*.png"))
print("Saved PNGs:", len(pngs))
print("Latest files:", [x.name for x in pngs[-6:]])
print("Folder:", PLOT_DIR)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 23.1 MB/s eta 0:00:00
Chosen test graphs: ['8OZ1_A_000435.npz', '7V62_A_000350.npz']
Saved PNGs: 4
Latest files: ['7V62_A_000350_baseline_top20.png', '7V62_A_000350_feat_v1_top20.png', '8OZ1_A_000435_baseline_top20.png', '8OZ1_A_000435_feat_v1_top20.png']
Folder: /content/drive/MyDrive/biolip_gnn/out/day16_pred_plots
